In [ ]:
import pandas as pd
import numpy as np

# A ML algorism used to predict which specialization an electerical engineering
# student would join in his third academic year depending on his grades in
# different subjects in the first two years of electrical engineering

In [ ]:
#MODELS :
from sklearn.ensemble import ExtraTreesClassifier
ext_1=ExtraTreesClassifier(random_state=1,n_estimators=400,max_depth=12)

from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(solver='liblinear',penalty='l2')

from xgboost.sklearn import XGBClassifier
xgb=XGBClassifier(random_state=42,n_estimators=200)

from sklearn.svm import SVC
svm=SVC(kernel='linear' , random_state=1,class_weight=weights)

from sklearn.ensemble import RandomForestClassifier

In [ ]:
df=pd.read_csv('/content/ntega - Sheet1 (6).csv')

In [ ]:
df.columns

Index(['ID', 'name', 'department', 'circuits 2', 'electronics 2',
       ' numerical analysis', 'measurements', 'digital design', 'machines 1-b',
       'electric tests b', 'fields2-b', 'hydro&thermo', 'total 1 ',
       'circuits 1', 'programming', 'material', 'math 2', 'fields 1',
       'electronics 1', 'tests 1', 'total 2'],
      dtype='object')

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

ID                     0
name                   0
department             0
circuits 2             0
electronics 2          0
 numerical analysis    0
measurements           0
digital design         0
machines 1-b           0
electric tests b       0
fields2-b              0
hydro&thermo           0
total 1                0
circuits 1             0
programming            0
material               0
math 2                 0
fields 1               0
electronics 1          0
tests 1                0
total 2                0
dtype: int64

In [ ]:
y=df.iloc[:,2].values
print(y.shape)
print(np.unique(y,return_counts=True))

(406,)
(array(['C', 'E', 'P'], dtype=object), array([ 75,  53, 278]))


In [ ]:
x=df.iloc[:,[4,5,7,10,12,14,20]].values
x.shape

(406, 7)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

In [ ]:
#acceptable F1score over all labels
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
ext=ExtraTreesClassifier(random_state=1)

scores=cross_val_score(
    estimator=ext,
    X=x,
    y=y,
    scoring=make_scorer(f1_score,average='weighted'),
    cv=LeaveOneOut()
)

In [ ]:
print(np.mean(scores))

0.7364532019704434


In [ ]:
# 0 F1score over E class :D , No Cases of E appear in the prediction
E_scores=cross_val_score(
    estimator=ext,
    X=x,
    y=y,
    scoring=make_scorer(f1_score,average='weighted',labels='E'),
    cv=LeaveOneOut()
)
print(np.mean(E_scores))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_cl

0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:
print(np.mean(E_scores))

0.0


In [ ]:
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
def cross_score_ros(x,y,estimator,sampling_strategy='auto'):
  kf = KFold(n_splits=x.shape[0])
  scores_smote=[]
  for fold, (train_index, test_index) in enumerate(kf.split(x), 1):
      X_train = x[train_index]
      y_train = y[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
      X_test = x[test_index]
      y_test = y[test_index]  # See comment on ravel and  y_train
      sm = SMOTE(random_state=42)
      X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
      model = estimator  # Choose a model here
      model.fit(X_train_oversampled, y_train_oversampled )
      y_pred = model.predict(X_test)

      #score=f1_score(y_test,y_pred,average='weighted')
      score=accuracy_score(y_test,y_pred)
      scores_smote.append(score)
  return scores_smote





In [ ]:
def cross_score_ros_label(x,y,estimator,labels,sampling_strategy='auto'):
  kf = KFold(n_splits=x.shape[0])
  scores_smote=[]
  for fold, (train_index, test_index) in enumerate(kf.split(x), 1):
      X_train = x[train_index]
      y_train = y[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
      X_test = x[test_index]
      y_test = y[test_index]  # See comment on ravel and  y_train
      sm = SMOTE(random_state=1,sampling_strategy=sampling_strategy)
      X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
      model = estimator  # Choose a model here
      model.fit(X_train_oversampled, y_train_oversampled )
      y_pred = model.predict(X_test)

      score=f1_score(y_test,y_pred,average='macro',labels=labels)#,pos_label=pos_label)
      #accuracy_score(y_test,y_pred)
      scores_smote.append(score)
  return scores_smote

In [ ]:
#using SMOTE for oversampling
#Overall F1 score is less

scores_smote=cross_score_ros(x,y,ext)
print(np.mean(scores_smote))





0.687192118226601


In [ ]:
E_scores_SMOTE=cross_score_ros_label(x=x,y=y,estimator=ext,labels='E')


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_cl

In [ ]:
print(np.mean(E_scores_SMOTE))
print(np.bincount(E_scores_SMOTE))

0.027093596059113302
[395  11]


In [ ]:
C_scores_SMOTE=cross_score_ros_label(x=x,y=y,estimator=ext,labels='C')

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_cl

In [ ]:
print(np.mean(C_scores_SMOTE))
print(np.bincount(C_scores_SMOTE))

0.116991643454039
[317  42]


In [ ]:
#using sampling_strategy parameter in SMOTE we can assign the number of
# samples per class in oversampling for every training sample
sampling_strategy={'E': 1000 , 'C':700 , 'P':278}
ss_E_scores_Smote=cross_score_ros_label(x=x,
                                         y=y,
                                         estimator=ext,
                                         labels='E',
                                         sampling_strategy=sampling_strategy)


/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (1000) in class E will be larger than the number of samples in the majority class (class #P -> 278)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (700) in class C will be larger than the number of samples in the majority class (class #P -> 278)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampl

In [ ]:
#better f1score for E is presented
print(np.mean(ss_E_scores_Smote))
print(np.bincount(ss_E_scores_Smote))

0.034482758620689655
[392  14]


In [ ]:
sampling_strategy={'E': 400 , 'C':400 , 'P':251}

In [ ]:

ss_C_scores_Smote=cross_score_ros_label(x=x,
                                         y=y,
                                         estimator=ext,
                                         labels='C',
                                         sampling_strategy=sampling_strategy)

/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (600) in class E will be larger than the number of samples in the majority class (class #P -> 251)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (600) in class C will be larger than the number of samples in the majority class (class #P -> 251)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (600) in class E will be larger than the number of samples in the majority class (class #P -> 251)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of sampl

KeyboardInterrupt: ignored

In [ ]:
print(np.mean(ss_C_scores_Smote))
print(np.bincount(ss_C_scores_Smote))

0.116991643454039
[317  42]


In [ ]:
overall_f1score_after_ss=cross_score_ros(x=x,
                                         y=y,
                                         estimator=ext,
                                         sampling_strategy=sampling_strategy)
print(np.mean(overall_f1score_after_ss))

0.687192118226601


In [ ]:
#overall f1_score decreased significantly
print(np.mean(overall_f1score_after_ss))

0.687192118226601


In [ ]:
from sklearn.metrics import accuracy_score
kf = KFold(n_splits=x.shape[0])
accuscores_smote=[]
for fold, (train_index, test_index) in enumerate(kf.split(x), 1):
  X_train = x[train_index]
  y_train = y[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
  X_test = x[test_index]
  y_test = y[test_index]  # See comment on ravel and  y_train
  sm = SMOTE(random_state=1)
  X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
  model = ext  # Choose a model here
  model.fit(X_train_oversampled, y_train_oversampled )
  y_pred = model.predict(X_test)

  score=accuracy_score(y_test,y_pred)
  accuscores_smote.append(score)

In [ ]:
#accuracy with sampling_strategy variable = 0.64
#accuracy without sampling_startegy variable=0.67
print(np.mean(accuscores_smote))

0.6945812807881774


In [ ]:
df=df.drop('name',axis=1)
df=df.drop('ID',axis=1)
df=df.drop('department',axis=1)

In [ ]:
df2.columns

Index(['circuits 2', 'electronics 2', ' numerical analysis', 'measurements',
       'digital design', 'machines 1-b', 'electric tests b', 'fields2-b',
       'hydro&thermo', 'total 1 ', 'circuits 1', 'programming', 'material',
       'math 2', 'fields 1', 'electronics 1', 'tests 1', 'total 2'],
      dtype='object')

In [ ]:
df2=df

In [ ]:
df2=df
#df2.drop(['hydro&thermo'],axis=1,inplace=True)
#df2.drop(['material'],axis=1,inplace=True)
#df2.drop(['circuits 2'],axis=1,inplace=True)
#df2.drop(['math 2'],axis=1,inplace=True)
df2.drop([' numerical analysis'],axis=1 , inplace=True)
df2.drop(['total 2'],axis=1,inplace=True)

In [ ]:
df2.corr()

,ID,circuits 2,electronics 2,numerical analysis,measurements,digital design,machines 1-b,electric tests b,fields2-b,hydro&thermo,total 1,circuits 1,programming,material,math 2,fields 1,electronics 1,tests 1,total 2
ID,1.000000,-0.016838,0.217600,0.066049,-0.020462,0.080255,0.224061,0.176033,-0.012815,0.033175,0.053534,0.035182,0.158057,-0.166530,-0.229451,-0.082892,0.106228,0.288315,0.097124
circuits 2,-0.016838,1.000000,0.683637,0.632426,0.604518,0.633887,0.655263,0.599040,0.657100,0.622649,0.220986,0.358414,0.549254,0.482522,0.580142,0.651898,0.583299,0.480205,0.828628
electronics 2,0.217600,0.683637,1.000000,0.703681,0.671422,0.707252,0.608351,0.751677,0.625717,0.657181,0.243785,0.282022,0.621274,0.485353,0.470745,0.648160,0.599425,0.599418,0.871687
numerical analysis,0.066049,0.632426,0.703681,1.000000,0.649858,0.585563,0.543991,0.601944,0.588071,0.685667,0.184246,0.232321,0.438124,0.379741,0.483609,0.584276,0.533070,0.448836,0.825497
measurements,-0.020462,0.604518,0.671422,0.649858,1.000000,0.593077,0.546294,0.633315,0.642441,0.647471,0.220316,0.249527,0.415821,0.435047,0.483991,0.580890,0.516405,0.461117,0.816228
digital design,0.080255,0.633887,0.707252,0.585563,0.593077,1.000000,0.498933,0.659394,0.639507,0.549869,0.197409,0.298345,0.609914,0.452235,0.485469,0.571519,0.566769,0.510925,0.796920
machines 1-b,0.224061,0.655263,0.608351,0.543991,0.546294,0.498933,1.000000,0.582905,0.634205,0.535376,0.236350,0.289121,0.371344,0.288258,0.442047,0.571951,0.505523,0.482731,0.755536
electric tests b,0.176033,0.599040,0.751677,0.601944,0.633315,0.659394,0.582905,1.000000,0.629161,0.567415,0.242197,0.323994,0.564351,0.451633,0.434678,0.636976,0.587452,0.647776,0.813909
fields2-b,-0.012815,0.657100,0.625717,0.588071,0.642441,0.639507,0.634205,0.629161,1.000000,0.624310,0.246851,0.268788,0.508186,0.494992,0.556948,0.676231,0.645351,0.536000,0.821171
hydro&thermo,0.033175,0.622649,0.657181,0.685667,0.647471,0.549869,0.535376,0.567415,0.624310,1.000000,0.190935,0.171936,0.455835,0.462704,0.445429,0.561421,0.528272,0.420496,0.810488


In [ ]:
df2.columns

Index(['ID', 'name', 'department', 'circuits 2', 'electronics 2',
       ' numerical analysis', 'measurements', 'digital design', 'machines 1-b',
       'electric tests b', 'fields2-b', 'hydro&thermo', 'total 1 ',
       'circuits 1', 'programming', 'material', 'math 2', 'fields 1',
       'electronics 1', 'tests 1', 'total 2'],
      dtype='object')

In [ ]:
#new approach , two level classification

x_full=df.iloc[:,[14,7]].values
x_full=x_full.astype('float64')


indices=[]
for i in range(0,y.shape[0]):
  if(y[i] != 'P'):
    indices.append(i)


y_new=y
y_new=np.where(y=='P','P','NP')
print(np.unique(y_new,return_counts=True))


(array(['NP', 'P'], dtype='<U2'), array([128, 278]))


In [ ]:


#first level of classification :
scores_1stlevel=cross_val_score(
    X=x_full,
    y=y_new,
    estimator=svm,
    cv=LeaveOneOut(),
    scoring=make_scorer(f1_score,labels=['NP','P'],pos_label='NP')
)
print(np.mean(scores_1stlevel))
#scored(0.1253)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_cl

0.16502463054187191


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:
print(np.mean(scores_1stlevel))

0.16502463054187191


In [ ]:
def cross_score_ros_pos_label(x,y,estimator,*labels,sampling_strategy='auto',pos_label=1):
  kf = KFold(n_splits=x.shape[0])
  scores_smote=[]
  for fold, (train_index, test_index) in enumerate(kf.split(x), 1):
      X_train = x[train_index]
      y_train = y[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
      X_test = x[test_index]
      y_test = y[test_index]  # See comment on ravel and  y_train
      sm = SMOTE(random_state=1,sampling_strategy={"P":400 ,"NP":700})
      X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
      model = estimator  # Choose a model here
      model.fit(X_train_oversampled, y_train_oversampled )
      y_pred = model.predict(X_test)

      score=f1_score(y_test,y_pred,average='binary',labels=labels,pos_label=pos_label)
      scores_smote.append(score)
  return scores_smote

In [ ]:
scores_1stlevel_smote=cross_score_ros_pos_label(
    x=x_full,
    y=y_new,
    estimator=svm,
    pos_label='NP'
)

/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (400) in class P will be larger than the number of samples in the majority class (class #P -> 278)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (700) in class NP will be larger than the number of samples in the majority class (class #P -> 278)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samples (400) in class P will be larger than the number of samples in the majority class (class #P -> 278)
  f"After over-sampling, the number of samples ({n_samples})"
/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:300: UserWarning: After over-sampling, the number of samp

In [ ]:
#without data engineering : 0.21674876847290642
print(np.mean(scores_1stlevel_smote))

0.22167487684729065


In [ ]:
kf = KFold(40)
preds=[]
scores_1st_ss=[]
for fold, (train_index, test_index) in enumerate(kf.split(x_full), 1):
   X_train = x_full[train_index]
   y_train = y_new[train_index]  # Based on your code, you might need a ravel call here, but I would look into how you're generating your y
   X_test = x_full[test_index]
   y_test = y_new[test_index]  # See comment on ravel and  y_train
   sm = SMOTE(random_state=1)#,sampling_strategy=sampling_strategy)
   X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
   model = svm # Choose a model here
   model.fit(X_train_oversampled, y_train_oversampled )
   y_pred = model.predict(X_test)
   score=f1_score(y_test,y_pred,pos_label='NP')
   scores_1st_ss.append(score)
   preds.append(y_pred)

In [ ]:
sampling_strategy={"P":300,"NP":700}

In [ ]:
##0.50 without weights , 0.54 after data engineering



print(np.mean(scores_1st_ss))

0.5353571428571429


In [ ]:
preds_arr=[]
for i in preds:
  for j in i:
    preds_arr.append(j)

In [ ]:
##[[118 160]
 ##[ 22 106]]
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
cfmat=confusion_matrix(y_new,preds_arr,labels=["P",'NP'])

y_new_enc=[]
preds_enc=[]
for i in range(y_new.shape[0]):
  if y_new[i] == 'P':
    y_new_enc.append(0)
  else :
    y_new_enc.append(1)

for i in preds_arr:
  if i == 'P':
    preds_enc.append(0)
  else :
    preds_enc.append(1)

print(cfmat)


print(roc_auc_score(y_new_enc,preds_enc))



[[227  51]
 [ 48  80]]
0.720773381294964


In [ ]:
# without weights :
## 118 160]
 ##[ 22 106

weights={"NP" : 3, "P" : 2}

In [ ]:
df2.columns

Index(['ID', 'name', 'department', 'circuits 2', 'electronics 2',
       ' numerical analysis', 'measurements', 'digital design', 'machines 1-b',
       'electric tests b', 'fields2-b', 'hydro&thermo', 'total 1 ',
       'circuits 1', 'programming', 'material', 'math 2', 'fields 1',
       'electronics 1', 'tests 1', 'total 2'],
      dtype='object')

In [ ]:
x_help=df2.iloc[:,[5,7,14,12,4]].values

In [ ]:
y_2nd=[]
x_2nd=[]

for i in indices:
  y_2nd.append(y[i])
  x_2nd.append(x_help[i])

In [ ]:
x_2nd=np.array(x_2nd)
y_2nd=np.array(y_2nd)
print(np.unique(y_2nd,return_counts=True))

(array(['C', 'E'], dtype='<U1'), array([75, 53]))


In [ ]:
svm_1=SVC(random_state=1,kernel='linear')
ext_2=ExtraTreesClassifier(random_state=1,n_estimators=500)
scores_2nd=cross_score_ros(

    estimator=ext_2,
    x=x_2nd,
    y=y_2nd
)

In [ ]:
print(np.mean(scores_2nd))

0.7109375
